In [1]:
pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# === SECTION 1: DATA CONVERSION & EMBEDDING PREP ===
# Convert structured JSON faculty data into content chunks for embedding

import json, os
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# === Config ===
DATA_FILE = "/content/drive/MyDrive/AI_Tutor_Phase2/faculty_data.json"
EMBEDDING_OUTPUT_PATH = "/content/drive/MyDrive/AI_Tutor_Phase2/faculty_faiss_index.bin"
METADATA_OUTPUT_PATH = "/content/drive/MyDrive/AI_Tutor_Phase2/faculty_metadata.json"

# === Load Data ===
with open(DATA_FILE, "r") as f:
    raw_data = json.load(f)

# === Load Embedding Model ===
bert_model = SentenceTransformer("all-MiniLM-L6-v2")

# === Process Data ===
metadata = []
texts = []

for prof in raw_data:
    try:
        content = f"""
Name: {prof['name']}
Designation: {prof['designation']}
Department: {prof['department']}
Email: {prof['email']}
Courses Taught: {', '.join(prof.get('courses_taught', []))}
Recent Publications: {', '.join(prof.get('recent_publications', []))}
Ongoing Projects: {prof.get('ongoing_projects', '')}
Research Interests: {prof.get('research_interests', '')}
Research Keywords: {', '.join(prof.get('research_keywords', []))}
""".strip()

        texts.append(content)
        metadata.append({
            "name": prof["name"],
            "designation": prof["designation"],
            "department": prof["department"],
            "email": prof["email"],
            "courses_taught": prof.get("courses_taught", []),
            "recent_publications": prof.get("recent_publications", []),
            "ongoing_projects": prof.get("ongoing_projects", ""),
            "research_interests": prof.get("research_interests", ""),
            "research_keywords": prof.get("research_keywords", []),
            "content": content
        })
    except Exception as e:
        print(f"❌ Error processing entry: {prof.get('name', 'Unknown')} - {e}")

# === Encode & Save ===
embeddings = bert_model.encode(texts, convert_to_numpy=True)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

faiss.write_index(index, EMBEDDING_OUTPUT_PATH)
with open(METADATA_OUTPUT_PATH, "w") as f:
    json.dump(metadata, f, indent=2)

print("✅ Data converted and index saved.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Data converted and index saved.


In [4]:
# === SECTION 2: Model Loading and Initialization ===

import os
import json
import faiss
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

# === Paths ===
drive_folder = "/content/drive/MyDrive/AI_Tutor_Phase2"
metadata_path = os.path.join(drive_folder, "faculty_metadata.json")
index_path = os.path.join(drive_folder, "faculty_faiss_index.bin")

# === Load Metadata ===
with open(metadata_path, "r") as f:
    metadata = json.load(f)

# === Load FAISS Index ===
faiss_index = faiss.read_index(index_path)

# === Load Sentence-BERT model for query embedding ===
bert_model = SentenceTransformer("all-MiniLM-L6-v2")

# === Load Phi model for generation ===
use_phi15 = False  # Set to True to use smaller, faster phi-1_5
phi_model_id = "microsoft/phi-1_5" if use_phi15 else "microsoft/phi-2"
phi_tokenizer = AutoTokenizer.from_pretrained(phi_model_id)
phi_model = AutoModelForCausalLM.from_pretrained(
    phi_model_id,
    device_map="auto",
    torch_dtype="auto"
)

# === Warmup Phi model to prevent cold-start lag ===
phi_model.generate(
    **phi_tokenizer("warmup", return_tensors="pt").to(phi_model.device),
    max_new_tokens=1
)

print("✅ Models and Index Loaded Successfully")


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


✅ Models and Index Loaded Successfully


In [ ]:
# === SECTION 3: Gradio Chat Interface ===
import gradio as gr

def rerank_results(query, chunks):
    priority = []
    query_lower = query.lower()

    for c in chunks:
        score = 0
        dept = c.get("department", "").lower()
        r_keywords = [kw.lower() for kw in c.get("research_keywords", [])]
        r_interests = c.get("research_interests", "").lower()
        projects = c.get("ongoing_projects", "").lower()
        courses = [c_name.lower() for c_name in c.get("courses_taught", []) if isinstance(c_name, str)]

        if any(word in query_lower for word in dept.split()):
            score += 1
        if any(kw in query_lower for kw in r_keywords):
            score += 1
        if r_interests and r_interests in query_lower:
            score += 1
        if any(cname in query_lower for cname in courses):
            score += 0.5
        if any(word in query_lower for word in projects.split()):
            score += 0.5

        priority.append((score, c))

    priority.sort(reverse=True, key=lambda x: x[0])

    # Debug print
    print("🔍 Top reranked candidates:")
    for score, chunk in priority[:5]:
        print(f"✔️ {chunk['name']} | Score: {score:.1f} | Dept: {chunk['department']}")

    return [x[1] for x in priority if x[0] > 0]

def truncate(text, max_words=400):
    return " ".join(text.split()[:max_words])

def build_context(top_chunks):
    return "\n\n".join(
        [f"{m['name']} ({m['designation']}, {m['department']}):\n{m['content']}" for m in top_chunks]
    )

def structured_query_response(query):
    query_lower = query.lower()
    results = []

    for prof in metadata:
        dept_match = any(word in prof.get("department", "").lower() for word in query_lower.split())
        research_match = any(word in prof.get("research_interests", "").lower() for word in query_lower.split()) or \
                         any(word in " ".join(prof.get("research_keywords", [])).lower() for word in query_lower.split())

        if "faculty" in query_lower and (dept_match or research_match):
            line = f"{prof['name']} ({prof['designation']}, {prof['department']}): {prof['research_interests']}"
            results.append(line)

    if results:
        return "Here are the relevant faculty:\n" + "\n".join("• " + r for r in results)
    else:
        return None  # Let LLM handle it

def gradio_chatbot(query, history=[]):
    print(f"\n🧠 Incoming query: {query}")
    q_embed = bert_model.encode([query], convert_to_numpy=True)
    D, I = faiss_index.search(q_embed, k=10)
    top_chunks = [metadata[i] for i in I[0]]

    reranked_chunks = rerank_results(query, top_chunks)
    top_chunks = reranked_chunks[:5] if reranked_chunks else top_chunks[:3]

    # Try structured fallback if confidence is low
    if not reranked_chunks or len(reranked_chunks) < 2:
        structured = structured_query_response(query)
        if structured:
            print("📦 Using structured query fallback.")
            return history + [(query, structured)]

    # Use LLM if reranked data is rich
    context = truncate(build_context(top_chunks))
    prompt = f"""You are a helpful academic assistant. Use the following context to answer.
Only respond based on the faculty data. Do not make up facts.

Context:
{context}

Question: {query}
Answer:"""

    inputs = phi_tokenizer(prompt, return_tensors="pt").to(phi_model.device)
    outputs = phi_model.generate(**inputs, max_new_tokens=300)
    raw_output = phi_tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = raw_output.split("Answer:")[-1].strip()

    return history + [(query, answer)]

# === Gradio App ===
with gr.Blocks() as demo:
    gr.Markdown("## 🎓 Faculty Research Chatbot\nAsk about departments, research, or projects")

    chatbox = gr.Chatbot()
    query_input = gr.Textbox(label="Ask your question...")
    clear = gr.Button("Clear Chat")

    query_input.submit(gradio_chatbot, inputs=query_input, outputs=chatbox)
    clear.click(lambda: [], None, chatbox)

demo.launch(debug=True, share=True)


/tmp/ipython-input-6-636091090.py:102: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbox = gr.Chatbot()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9645505fcff1a33dfa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🧠 Incoming query: Who teaches data structures?
🔍 Top reranked candidates:
✔️ Dr. Ayesha Nair | Score: 0.5 | Dept: Computer Science
✔️ Dr. Chloe Nguyen | Score: 0.5 | Dept: Statistics
✔️ Dr. Marcus Lee | Score: 0.0 | Dept: Information Systems
✔️ Dr. Thomas Nguyen | Score: 0.0 | Dept: Computer Science
✔️ Dr. Alan Matthews | Score: 0.0 | Dept: Mathematics


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...


🧠 Incoming query: Can you provide a brief summary about the project?
🔍 Top reranked candidates:
✔️ Dr. Sara Gomez | Score: 0.0 | Dept: Chemistry
✔️ Dr. Rajeev Bansal | Score: 0.0 | Dept: Physics
✔️ Dr. Daniel Harper | Score: 0.0 | Dept: History
✔️ Dr. Priya Khanna | Score: 0.0 | Dept: Biology
✔️ Dr. Julia Park | Score: 0.0 | Dept: English


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🧠 Incoming query: Can you provide a detailed summary about the project?
🔍 Top reranked candidates:
✔️ Dr. Meera Rajan | Score: 0.5 | Dept: Political Science
✔️ Dr. Sara Gomez | Score: 0.0 | Dept: Chemistry
✔️ Dr. Rajeev Bansal | Score: 0.0 | Dept: Physics
✔️ Dr. Omar Patel | Score: 0.0 | Dept: Cybersecurity
✔️ Dr. Daniel Harper | Score: 0.0 | Dept: History


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🧠 Incoming query: generate 5 QA for exam preparation on deeplearning
🔍 Top reranked candidates:
✔️ Dr. James Carter | Score: 1.5 | Dept: Psychology
✔️ Dr. Ayesha Nair | Score: 0.5 | Dept: Computer Science
✔️ Dr. Thomas Nguyen | Score: 0.5 | Dept: Computer Science
✔️ Dr. Alan Matthews | Score: 0.5 | Dept: Mathematics
✔️ Dr. Priya Khanna | Score: 0.5 | Dept: Biology


In [ ]:
# # === SECTION 3: Gradio Chat Interface ===
# import gradio as gr

# def rerank_results(query, chunks):
#     priority = []

#     for c in chunks:
#         score = 0
#         q = query.lower()

#         if any(word in q for word in c.get("department", "").lower().split()):
#             score += 1
#         if any(kw.lower() in q for kw in c.get("research_keywords", [])):
#             score += 1
#         if c.get("research_interests", "").lower() in q:
#             score += 1
#         if any(kw in q for kw in c.get("courses_taught", [])):
#             score += 0.5
#         if any(word in q for word in c.get("ongoing_projects", "").lower().split()):
#             score += 0.5

#         priority.append((score, c))

#     # Sort by score descending
#     priority.sort(reverse=True, key=lambda x: x[0])
#     return [x[1] for x in priority if x[0] > 0]

#     print("🔍 Top candidates post-rerank:")
#     for score, chunk in priority[:5]:
#         print(f"✔️ {chunk['name']} | Score: {score} | Department: {chunk['department']}")

# def truncate(text, max_words=400):
#     return " ".join(text.split()[:max_words])

# def build_context(top_chunks):
#     return "\n\n".join(
#         [f"{m['name']} ({m['designation']}, {m['department']}):\n{m['content']}" for m in top_chunks]
#     )

# def structured_query_response(query):
#     query_lower = query.lower()
#     results = []

#     for prof in metadata:
#         dept_match = any(word in prof.get("department", "").lower() for word in query_lower.split())
#         research_match = any(word in prof.get("research_interests", "").lower() for word in query_lower.split()) \
#             or any(word in " ".join(prof.get("research_keywords", [])).lower() for word in query_lower.split())

#         if "faculty" in query_lower and (dept_match or research_match):
#             line = f"{prof['name']} ({prof['designation']}, {prof['department']}): {prof['research_interests']}"
#             results.append(line)

#     if results:
#         return "Here are the relevant faculty:\n" + "\n".join("• " + r for r in results)
#     else:
#         return "I'm unable to fetch any matching faculty from the data."



# def gradio_chatbot(query, history=[]):
#     q_embed = bert_model.encode([query], convert_to_numpy=True)
#     D, I = faiss_index.search(q_embed, k=10)
#     top_chunks = [metadata[i] for i in I[0]]

#     reranked_chunks = rerank_results(query, top_chunks)
#     top_chunks = reranked_chunks[:5] if reranked_chunks else top_chunks[:3]


#     # Smart fallback to structured logic
#     if "faculty" in query.lower() or "professor" in query.lower() or "research" in query.lower():
#         structured_output = structured_query_response(query)
#         if "unable to fetch" not in structured_output.lower():
#             return history + [(query, structured_output)]

#     context = truncate(build_context(top_chunks))
#     prompt = f"""You are a helpful academic assistant. Use the following context to answer.
# Only respond based on the faculty data. Do not make up facts.

# Context:
# {context}

# Question: {query}
# Answer:"""

#     inputs = phi_tokenizer(prompt, return_tensors="pt").to(phi_model.device)
#     outputs = phi_model.generate(**inputs, max_new_tokens=300)
#     raw_output = phi_tokenizer.decode(outputs[0], skip_special_tokens=True)
#     answer = raw_output.split("Answer:")[-1].strip()

#     return history + [(query, answer)]

# # === Gradio App ===
# with gr.Blocks() as demo:
#     gr.Markdown("## 🎓 Faculty Research Chatbot\nAsk about departments, research, or projects")

#     chatbox = gr.Chatbot()
#     query_input = gr.Textbox(label="Ask your question...")
#     clear = gr.Button("Clear Chat")

#     query_input.submit(gradio_chatbot, inputs=query_input, outputs=chatbox)
#     clear.click(lambda: [], None, chatbox)

# demo.launch(debug=True, share=True)


In [ ]:
# === SECTION 3: Gradio Chat Interface ===
import gradio as gr

# === Utility: Reranking Function ===
def rerank_results(query, chunks):
    priority = []
    query_lower = query.lower()

    for c in chunks:
        score = 0
        dept = c.get("department", "").lower()
        r_keywords = [kw.lower() for kw in c.get("research_keywords", [])]
        r_interests = c.get("research_interests", "").lower()
        projects = c.get("ongoing_projects", "").lower()
        courses = [c_name.lower() for c_name in c.get("courses_taught", []) if isinstance(c_name, str)]

        if any(word in query_lower for word in dept.split()):
            score += 1
        if any(kw in query_lower for kw in r_keywords):
            score += 1
        if r_interests and r_interests in query_lower:
            score += 1
        if any(cname in query_lower for cname in courses):
            score += 0.5
        if any(word in query_lower for word in projects.split()):
            score += 0.5

        priority.append((score, c))

    priority.sort(reverse=True, key=lambda x: x[0])

    print("🔍 Top reranked candidates:")
    for score, chunk in priority[:5]:
        print(f"✔️ {chunk['name']} | Score: {score:.1f} | Dept: {chunk['department']}")

    return [x[1] for x in priority if x[0] > 0]

# === Utility: Text Builders ===
def truncate(text, max_words=400):
    return " ".join(text.split()[:max_words])

def build_context(top_chunks):
    return "\n\n".join(
        [f"{m['name']} ({m['designation']}, {m['department']}):\n{m['content']}" for m in top_chunks]
    )

def structured_query_response(query):
    query_lower = query.lower()
    results = []

    for prof in metadata:
        dept_match = any(word in prof.get("department", "").lower() for word in query_lower.split())
        research_match = any(word in prof.get("research_interests", "").lower() for word in query_lower.split()) or \
                         any(word in " ".join(prof.get("research_keywords", [])).lower() for word in query_lower.split())

        if "faculty" in query_lower and (dept_match or research_match):
            line = f"{prof['name']} ({prof['designation']}, {prof['department']}): {prof['research_interests']}"
            results.append(line)

    if results:
        return "Here are the relevant faculty:\n" + "\n".join("• " + r for r in results)
    else:
        return None  # Let LLM handle it

# === Main Chat Function ===
def gradio_chatbot(query, history=[]):
    print(f"\n🧠 Incoming query: {query}")
    general_keywords = ["create questions", "generate qa", "mcq", "exam", "interview questions", "question and answer", "prepare questions"]

    # === Handle general Q&A queries with Phi-2's own pretrained knowledge
    if any(kw in query.lower() for kw in general_keywords):
        print("🧠 Using general knowledge mode (no FAISS context)")
        prompt = f"""You are a helpful AI tutor. Generate 10 exam-style questions and answers for:

{query}

Answer:"""
        inputs = phi_tokenizer(prompt, return_tensors="pt").to(phi_model.device)
        outputs = phi_model.generate(**inputs, max_new_tokens=600)
        raw_output = phi_tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = raw_output.split("Answer:")[-1].strip()
        return history + [(query, answer)]

    # === FAISS Retrieval
    q_embed = bert_model.encode([query], convert_to_numpy=True)
    D, I = faiss_index.search(q_embed, k=10)
    top_chunks = [metadata[i] for i in I[0]]

    # === Rerank
    reranked_chunks = rerank_results(query, top_chunks)
    top_chunks = reranked_chunks[:5] if reranked_chunks else top_chunks[:3]

    # === Try rule-based answer if confidence is low
    if not reranked_chunks or len(reranked_chunks) < 2:
        structured = structured_query_response(query)
        if structured:
            print("📦 Using structured query fallback.")
            return history + [(query, structured)]

    # === LLM-based Answer from faculty context
    context = truncate(build_context(top_chunks))
    prompt = f"""You are a helpful academic assistant. Use the following context to answer.
Only respond based on the faculty data. Do not make up facts.

Context:
{context}

Question: {query}
Answer:"""

    inputs = phi_tokenizer(prompt, return_tensors="pt").to(phi_model.device)
    outputs = phi_model.generate(**inputs, max_new_tokens=300)
    raw_output = phi_tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = raw_output.split("Answer:")[-1].strip()

    return history + [(query, answer)]

# === Gradio Interface ===
with gr.Blocks() as demo:
    gr.Markdown("## 🎓 Faculty Research Chatbot\nAsk about departments, research, or projects")

    chatbox = gr.Chatbot()
    query_input = gr.Textbox(label="Ask your question...")
    clear = gr.Button("Clear Chat")

    query_input.submit(gradio_chatbot, inputs=query_input, outputs=chatbox)
    clear.click(lambda: [], None, chatbox)

demo.launch(debug=True, share=True)
